In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName('dataincode') \
    .config("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.6.0.jar") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.spark_catalog.type", "hive") \
    .config("spark.sql.catalog.local.warehouse", "s3a://datalake/iceberg") \
    .getOrCreate()

#Ajuste de log WARN log para ERROR
spark.sparkContext.setLogLevel("ERROR")

24/11/05 22:35:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Atualizações em Nível de Linha no Apache Iceberg

O Apache Iceberg gerencia atualizações em nível de linha por meio de dois modos principais: **copy-on-write (COW)** e **merge-on-read (MOR)**.

**Copy-on-Write (COW):** COW é o modo padrão no Iceberg. Quando registros na tabela são alterados (atualizados ou excluídos), os arquivos de dados associados a esses registros são reescritos com as alterações aplicadas. Este modo oferece as leituras mais rápidas, mas tem as atualizações e exclusões mais lentas.

**Merge-on-Read (MOR):** No MOR, as atualizações e exclusões não resultam na reescrita imediata de arquivos de dados. Em vez disso, "arquivos deletados" são criados para registrar as alterações. Esses arquivos deletados são então usados durante as operações de leitura para filtrar ou atualizar os dados afetados. O MOR geralmente oferece leituras e gravações rápidas. No entanto, o uso regular de compactação é recomendado para minimizar os custos de leitura, pois vários arquivos de exclusão podem se acumular ao longo do tempo, levando a operações de leitura mais complexas.

**Tipos de arquivo de exclusão:** Existem dois tipos de arquivos de exclusão: arquivos de exclusão posicional e arquivos de exclusão de igualdade. A escolha do tipo de arquivo de exclusão afeta o desempenho de leitura e gravação.

**Caso de Uso :** Se as leituras forem mais frequentes que as gravações, o COW pode ser mais adequado. Se as gravações forem frequentes, o MOR pode ser preferível.

**Fontes:**

https://iceberg.apache.org/docs/1.5.2/configuration/#write-properties<br>
Apache Iceberg: The Definitive Guide<br>
https://aws.amazon.com/pt/blogs/big-data/use-apache-iceberg-in-a-data-lake-to-support-incremental-data-processing/?ref=guptaakashdeep.com


####  <span style="color:blue">COW:</span>
Sempre que uma linha em um arquivo de dados é atualizada ou excluída, todo o arquivo de dados é reescrito com as alterações aplicadas. Isso significa que mesmo se apenas uma única linha for alterada, um novo arquivo de dados será criado, e o antigo será substituído

**Vantagens:** Leitura rápida, pois o leitor só precisa ler os dados sem mesclá-los com arquivos excluídos ou atualizados.

**Desvantagens:** Gravação lenta, pois a reescrita de arquivos de dados inteiros pode ser um processo demorado, especialmente se houver atualizações frequentes.

#### Cenários de uso ideais para COW:
**Priorizando leituras rápidas:** Se o desempenho de leitura for crítico e as gravações puderem ser um pouco mais lentas, COW é uma boa opção. Em resumo mais ```INSERT``` menos ```UPDATE, DELETE```


#### <span style="color:blue">MOR:</span>
As atualizações não resultam na reescrita imediata de arquivos de dados inteiros. Em vez disso, as alterações são rastreadas em arquivos de exclusão separados.

**Exclusão de um registro:** O registro é listado em um arquivo de exclusão. Quando o leitor lê a tabela, ele mescla os dados com o arquivo de exclusão para decidir qual registro ignorar.

**Atualização de um registro:** O registro modificado também é rastreado em um arquivo de exclusão, e o mecanismo cria um novo arquivo de dados contendo o registro com o valor atualizado. Ao ler a tabela, o mecanismo ignora a versão antiga do registro por causa do arquivo de exclusão e usa a nova versão no novo arquivo de dados.

**Vantagens:** Gravação mais rápida em comparação com COW, pois não há necessidade de reescrever arquivos de dados inteiros.

**Desvantagens:** Leitura mais lenta em comparação com COW, pois os arquivos de exclusão precisam ser reconciliados durante a leitura.

#### Cenários de uso ideais:

**Tabelas com muitas atualizações:** Se a tabela for atualizada com frequência, MOR pode ser uma boa escolha, pois pode acelerar as gravações.

**Priorizando gravações rápidas:** Se o desempenho de gravação for crítico e as leituras puderem ser um pouco mais lentas, MOR pode ser uma boa opção.

**Compactação:** A compactação regular é essencial ao usar MOR para minimizar o custo de leitura de dados


#### <span style="color:blue">Como habilitar o MOR:</span>

**Propriedades:**
- ```write.delete.mode```: Abordagem usada para transações de exclusão.
- ```write.update.mode```: Abordagem usada para transações de atualização.
- ```write.merge.mode```:  Abordagem usada para transações de mesclagem.
---
> ⚠️ O correto funcionamento vai depender da compatibilidade da  mecanismo de computação, neste caso estamos usando o SPARK.
---

**Tabela já existente:**

```sql
ALTER TABLE catalog.db.nome_da_tabela SET TBLPROPERTIES (
'write.delete.mode'='merge-on-read',
'write.update.mode'='merge-on-read',
'write.merge.mode'='merge-on-read'
)
```

**Nova tabela:**

```sql
CREATE TABLE catalog.db.nome_da_tabela (
    id int,
    first_name string,
    last_name string
) TBLPROPERTIES (
    'write.delete.mode'='copy-on-write',
    'write.update.mode'='merge-on-read',
    'write.merge.mode'='merge-on-read'
) USING iceberg
```

In [6]:
# spark.sql("SHOW CATALOG").show()
#Remover configurações
##ALTER TABLE prod.db.sample UNSET TBLPROPERTIES ('read.split.target-size');
spark.sql("""
ALTER TABLE iceberg.bronze.vendas SET TBLPROPERTIES (
'write.delete.mode'='copy-on-write',
'write.update.mode'='merge-on-read',
'write.merge.mode'='merge-on-read'
)""")

DataFrame[]

In [7]:
extended_info = spark.sql("DESCRIBE EXTENDED iceberg.bronze.vendas")

table_properties = extended_info.filter(extended_info["col_name"] == "Table Properties").take(1)[0][1]

properties = table_properties.strip('[]').split(',')

for item in properties:
    print(f"{item}")

current-snapshot-id=143286272759386465
format=iceberg/parquet
format-version=2
write.delete.mode=copy-on-write
write.merge.mode=merge-on-read
write.parquet.compression-codec=zstd
write.update.mode=merge-on-read


In [8]:
spark.stop()